In [1]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
    drive.mount('/content/gdrive')
    print('Google Drive is mounted\n')
else:
    print('Google Drive is already mounted\n')
  

Google Drive is already mounted



In [2]:
if os.path.exists('/content/gdrive/My Drive/Colab Notebooks/data')==True:
    #print('Train data downloading..')
    #! curl 'https://raw.githubusercontent.com/unizard/2019.Spring.AI/master/data-zoo-v2.csv' -o '/content/gdrive/My Drive/Colab Notebooks/data-zoo-v2.csv'
    print('Done..\n')
#else:
    #print('File already exists \n')

Done..



In [3]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import array
 
TRAIN_DIR = '/content/gdrive/My Drive/Colab Notebooks/data'
train_folder_list = array(os.listdir('/content/gdrive/My Drive/Colab Notebooks/data'))
 
train_input = []
train_label = []
 
label_encoder = LabelEncoder()  # LabelEncoder Class 호출
integer_encoded = label_encoder.fit_transform(train_folder_list)
onehot_encoder = OneHotEncoder(sparse=False) 
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
 
for index in range(len(train_folder_list)):
    path = os.path.join(TRAIN_DIR, train_folder_list[index])
    path = path + '/'
    img_list = os.listdir(path)
    for img in img_list:
        img_path = os.path.join(path, img)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        train_input.append([np.array(img)])
        train_label.append([np.array(onehot_encoded[index])])
 
train_input = np.reshape(train_input, (-1, 784))
train_label = np.reshape(train_label, (-1, 10))
train_input = np.array(train_input).astype(np.float32)
train_label = np.array(train_label).astype(np.float32)
np.save("train_data.npy", train_input)
np.save("train_label.npy", train_label)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

print(train_input)
train_input= MinMaxScaler().fit_transform(train_input)


print(train_input)
"""
mean = train_input.mean(axis=0)
std = train_input.std(axis=0)
train_input = (train_input - mean) / std

print(train_input)

"""



[[ 63.  73.  62. ...  98.  90.  87.]
 [180. 183. 194. ...  70.  46.  42.]
 [149. 166. 169. ... 118. 127. 116.]
 ...
 [143. 135. 140. ... 166. 126.  46.]
 [186. 190. 206. ... 249. 252. 253.]
 [213. 214. 215. ... 186. 184. 183.]]
[[0.24705884 0.28627452 0.24313727 ... 0.38431376 0.3529412  0.34117648]
 [0.7058824  0.7176471  0.7607844  ... 0.27450982 0.18039216 0.16470589]
 [0.58431375 0.6509804  0.6627451  ... 0.46274513 0.49803925 0.454902  ]
 ...
 [0.56078434 0.5294118  0.54901963 ... 0.6509804  0.49411768 0.18039216]
 [0.7294118  0.74509805 0.8078432  ... 0.97647065 0.98823535 0.9921569 ]
 [0.8352942  0.83921576 0.8431373  ... 0.7294118  0.72156864 0.7176471 ]]


'\nmean = train_input.mean(axis=0)\nstd = train_input.std(axis=0)\ntrain_input = (train_input - mean) / std\n\nprint(train_input)\n\n'

In [5]:

import random
tmp = [[x,y] for x, y in zip(train_input, train_label)]
random.shuffle(tmp)
data = np.reshape(tmp,(-1)) 
x_train = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]

train_input = np.reshape(x_train, (-1, 784))
train_label = np.reshape(y_train, (-1, 10))

train_input=train_input[:10000]
train_label=train_label[:10000]
print(train_input.shape)
print(train_label.shape)



(10000, 784)
(10000, 10)


In [6]:
learning_rate=1e-3
training_epochs=60
batch_size=100

keep_prob=tf.placeholder(tf.float32)

X=tf.placeholder(tf.float32,[None,784])
X_img=tf.reshape(X,[-1,28,28,1])
Y=tf.placeholder(tf.float32,[None,10])

W1=tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1=tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L1=tf.nn.dropout(L1,keep_prob=keep_prob)

W2=tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L21=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L2=tf.nn.dropout(L2,keep_prob=keep_prob)

W3=tf.Variable(tf.random_normal([3,3,64,128],stddev=0.01))
L3=tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
L3=tf.nn.relu(L3)
L3=tf.nn.max_pool(L3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L3=tf.nn.dropout(L3,keep_prob=keep_prob)
flat_L3=tf.reshape(L3,[-1,4*4*128])

W4=tf.get_variable('W4',shape=[4*4*128,625],initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.random_normal([625]))
L4=tf.nn.relu(tf.matmul(flat_L3,W4)+b4)
L4=tf.nn.dropout(L4,keep_prob=keep_prob)

W5=tf.get_variable('W5',shape=[625,10],initializer=tf.contrib.layers.xavier_initializer())
b5=tf.Variable(tf.random_normal([10]))

logits=tf.matmul(L4,W5)+b5
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
sess=tf.Session()
sess.run(tf.global_variables_initializer())



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
x_batch=train_input
y_batch=train_label

print(x_batch.shape)
print(y_batch.shape)

print('start')
for epoch in range(training_epochs):
  avg_cost=0
  a=0
  total_batch=int(10000/batch_size)
  for i in range(total_batch):
    
    c,_=sess.run([cost,optimizer],feed_dict={X:x_batch[a:a+batch_size],Y:y_batch[a:a+batch_size],keep_prob:0.7})
    a = a + batch_size

    avg_cost+=c/total_batch
  print('Epoch:','%04d' %(epoch+1),'cost=','{:.9f}'.format(avg_cost))
print('finish')


(10000, 784)
(10000, 10)
start
Epoch: 0001 cost= 2.421127188
Epoch: 0002 cost= 2.212728181
Epoch: 0003 cost= 2.070234253
Epoch: 0004 cost= 1.953218075
Epoch: 0005 cost= 1.856250610
Epoch: 0006 cost= 1.765561577
Epoch: 0007 cost= 1.679401360
Epoch: 0008 cost= 1.612296209
Epoch: 0009 cost= 1.548886586
Epoch: 0010 cost= 1.491718993
Epoch: 0011 cost= 1.427220143
Epoch: 0012 cost= 1.361480595
Epoch: 0013 cost= 1.320330094
Epoch: 0014 cost= 1.272182001
Epoch: 0015 cost= 1.213175397
Epoch: 0016 cost= 1.159733145
Epoch: 0017 cost= 1.119609242
Epoch: 0018 cost= 1.080214397
Epoch: 0019 cost= 1.061179734
Epoch: 0020 cost= 1.001228926
Epoch: 0021 cost= 0.955987942
Epoch: 0022 cost= 0.932799187
Epoch: 0023 cost= 0.885009645
Epoch: 0024 cost= 0.852186059
Epoch: 0025 cost= 0.842143604
Epoch: 0026 cost= 0.813428723
Epoch: 0027 cost= 0.800975687
Epoch: 0028 cost= 0.770998178
Epoch: 0029 cost= 0.733774040
Epoch: 0030 cost= 0.704684474
Epoch: 0031 cost= 0.681035911
Epoch: 0032 cost= 0.667236988
Epoch: 00